# Import data

In [128]:
import numpy as np
import pandas as pd
import json

In [129]:
file = '../raw_data/full_dump.json'
with open(file) as data_file:    
    data = json.load(data_file) 
len(data)

35320

In [130]:
data_keys = list(data.keys())
data_keys[:5]

['EU Challenger Series/2017 Season/Spring Qualifiers/Scoreboards_1_1',
 'EU Challenger Series/2017 Season/Spring Qualifiers/Scoreboards_2_1',
 'EU Challenger Series/2017 Season/Spring Qualifiers/Scoreboards_3_1',
 'EU Challenger Series/2017 Season/Spring Qualifiers/Scoreboards_4_1',
 'EU Challenger Series/2017 Season/Spring Qualifiers/Scoreboards_5_1']

In [131]:
import re

pattern = r'(\d{4})'
re.search(pattern, data_keys[2]).group(1)

year = []
no_year = []
for i in range(len(data_keys)):
    try:
        year.append(re.search(pattern, data_keys[i]).group(1))
    except:
        no_year.append(data_keys[i])

np.unique(year), len(no_year)

(array(['2016', '2017', '2018', '2019', '2020', '2021'], dtype='<U4'), 1703)

# Normalize the JSON file

Normalize means to expand the nested keys in the JSON file and the subsequent DataFrame.

In [132]:
#Normalize the complete dataframe and make a copy
df_normalized_teams = pd.json_normalize(data.values())
df_teams = df_normalized_teams.copy()
df_teams.sample(5)

,id,start,patch,winner,duration,picks_bans,teams.BLUE.name,teams.BLUE.total_turret_kills,teams.BLUE.total_inhibitor_kills,teams.BLUE.total_rift_herald_kills,...,teams.BLUE.side,teams.BLUE.players,teams.RED.name,teams.RED.total_turret_kills,teams.RED.total_inhibitor_kills,teams.RED.total_rift_herald_kills,teams.RED.total_dragon_kills,teams.RED.total_baron_kills,teams.RED.side,teams.RED.players
730,LCK/2017 Season/Spring Season/Scoreboards/Week...,None,7.2,BLUE,2944,"[{'champion_name': 'Malzahar', 'is_ban': True}...",Kongdoo Monster,11,5,0,...,BLUE,"[{'name': 'Roach', 'id': 196336, 'role': 'TOP'...",Jin Air Green Wings,3,0,0,4,0,RED,"[{'name': 'SoHwan', 'id': 198047, 'role': 'TOP..."
5465,TCS/2018 Season/Winter Season/Scoreboards/Week...,None,8.2,BLUE,2096,"[{'champion_name': 'Sejuani', 'is_ban': True},...",Oyun Hizmetleri,10,3,1,...,BLUE,"[{'name': 'Smoke', 'id': 197951, 'role': 'TOP'...",Macro Maniacs,5,0,0,1,0,RED,"[{'name': 'Cube', 'id': 145454, 'role': 'TOP',..."
21674,2020 Mid-Season Cup/Scoreboards/Knockout Stage...,None,10.10,BLUE,1985,"[{'champion_name': 'Ornn', 'is_ban': True}, {'...",FunPlus Phoenix,11,4,2,...,BLUE,"[{'name': 'Khan', 'id': 171549, 'role': 'TOP',...",JD Gaming,2,0,0,0,0,RED,"[{'name': 'Zoom', 'id': 206627, 'role': 'TOP',..."
14961,UK League Championship/2019 Season/Summer Seas...,None,9.13,RED,2407,"[{'champion_name': 'Irelia', 'is_ban': True}, ...",Barrage (British Team),3,0,1,...,BLUE,"[{'name': 'Zhergoth', 'id': 206471, 'role': 'T...",MnM Gaming,11,3,0,6,1,RED,"[{'name': 'Only Angel', 'id': 258380, 'role': ..."
5407,LLN/2018 Season/Opening Season/Scoreboards/Wee...,None,8.2,RED,2820,"[{'champion_name': 'Jarvan IV', 'is_ban': True...",Dash9 Gaming,4,0,0,...,BLUE,"[{'name': 'Akunma', 'id': 189175, 'role': 'TOP...",Rainbow7,11,4,1,3,2,RED,"[{'name': 'Jirall', 'id': 169983, 'role': 'TOP..."


In [133]:
#Make a dataframe dedicated to the teams 
df_teams.drop(['teams.BLUE.players', 'teams.RED.players', 'picks_bans'], axis=1, inplace=True)
df_teams.head()

,id,start,patch,winner,duration,teams.BLUE.name,teams.BLUE.total_turret_kills,teams.BLUE.total_inhibitor_kills,teams.BLUE.total_rift_herald_kills,teams.BLUE.total_dragon_kills,teams.BLUE.total_baron_kills,teams.BLUE.side,teams.RED.name,teams.RED.total_turret_kills,teams.RED.total_inhibitor_kills,teams.RED.total_rift_herald_kills,teams.RED.total_dragon_kills,teams.RED.total_baron_kills,teams.RED.side
0,EU Challenger Series/2017 Season/Spring Qualif...,None,6.24,BLUE,2023,Fnatic Academy,11,3,0,2,1,BLUE,EURONICS Gaming,4,0,0,1,0,RED
1,EU Challenger Series/2017 Season/Spring Qualif...,None,6.24,RED,1782,AlienTech eSports,3,0,0,0,0,BLUE,Team Larssen,7,1,1,2,1,RED
2,EU Challenger Series/2017 Season/Spring Qualif...,None,6.24,RED,2465,Bobaski,4,0,0,0,0,BLUE,Nerv,9,1,0,3,1,RED
3,EU Challenger Series/2017 Season/Spring Qualif...,None,6.24,BLUE,1925,Team Forge,11,4,0,4,1,BLUE,Tricked Esport,0,0,0,0,1,RED
4,EU Challenger Series/2017 Season/Spring Qualif...,None,6.24,BLUE,1507,Team-LDLC,11,3,0,1,1,BLUE,ASUS ROG Army,1,0,0,0,0,RED


In [134]:
#Make a dataframe dedicated to the team BLUE
df_normalized_teams['teams.BLUE.players'].explode()
df_normalized_BLUE = pd.json_normalize(df_normalized_teams['teams.BLUE.players'].explode())
df_BLUE = df_normalized_BLUE.copy()
df_BLUE.sample(5)

,name,id,role,champion_name,champion_id,gold_15,kills_assists_15,deaths_15,total_gold,total_cs,total_kills,total_monster_kills,total_assists,total_deaths,total_damage_taken,total_damage_dealt,win,side
69100,Kartis,298207.0,TOP,Camille,164,4700.0,11.0,0.0,17360,348,5,56,6,0,25666,291749,True,BLUE
146349,Raxxo,194896.0,SUP,Tahm Kench,223,4968.0,12.0,0.0,12602,257,0,10,12,0,32587,131188,True,BLUE
160722,Regi,194987.0,MID,Zed,238,4395.0,2.0,4.0,14057,292,2,22,0,4,19067,212463,False,BLUE
149159,Gralou,162968.0,SUP,Rell,526,3491.0,11.0,2.0,8927,40,4,0,7,2,12000,31011,True,BLUE
22738,Breeze,193045.0,BOT,Xayah,498,5148.0,13.0,0.0,15305,325,7,2,6,0,9042,221452,True,BLUE


In [135]:
#Make a dataframe dedicated to the team RED
df_normalized_teams['teams.RED.players'].explode()
df_normalized_RED = pd.json_normalize(df_normalized_teams['teams.RED.players'].explode())
df_RED = df_normalized_RED.copy()
df_RED.sample(5)

,name,id,role,champion_name,champion_id,gold_15,kills_assists_15,deaths_15,total_gold,total_cs,total_kills,total_monster_kills,total_assists,total_deaths,total_damage_taken,total_damage_dealt,win,side
163034,Regret9,195010.0,SUP,Morgana,25,3075.0,2.0,4.0,5229,17,0,0,2,4,8359,16637,False,RED
42517,None,NaN,MID,Corki,42,4480.0,5.0,8.0,6283,117,1,0,4,8,16391,50298,False,RED
104363,Ruvelius,196533.0,BOT,Gnar,150,5108.0,3.0,5.0,10448,232,2,20,1,5,25911,145590,False,RED
85025,Chevis,272605.0,TOP,Irelia,39,5020.0,14.0,5.0,15273,277,8,8,6,5,22265,186774,True,RED
158085,BalIs,191215.0,TOP,Fiora,114,6157.0,10.0,0.0,12687,246,4,13,6,0,20701,138997,True,RED


In [136]:
#Make a dataframe dedicated to the Bans
df_normalized_teams['picks_bans'].explode()
df_normalized_BANS = pd.json_normalize(df_normalized_teams['picks_bans'].explode())
df_BANS = df_normalized_BANS.copy()
df_BANS.isnull().sum()/len(df_BANS)

champion_name    0.019285
is_ban           0.019285
dtype: float64

##  Include the game id to match later different DataFrames

We have two different dataframes for team Blue and team Red, later, we can merge them on the game id. This will also be useful when we want to come back for the y parameter! 

In [137]:
get_index = df_normalized_teams['id'].tolist()
index_preproc = np.asarray([[index] * 5 for index in get_index])
index_teams = index_preproc.reshape(len(df_normalized_teams) * 5).tolist()
df_RED['game_id'] = index_teams
df_BLUE['game_id'] = index_teams

# Filter years that are going to be analyzed

In [138]:
# Get the years and the number of games played
df_normalized_teams['start'] = pd.to_datetime(df_normalized_teams['start'])
df_normalized_teams['year'] = pd.DatetimeIndex(df_normalized_teams['start']).year
games_year = df_normalized_teams.year.unique()

for year in games_year:
    N_games = len(df_normalized_teams[df_normalized_teams['year'] == year])
    print(f'Number of games in {year}: {N_games}')

Number of games in nan: 0


In [139]:
# Function that returns a Dataframe just with the years of interest
def choose_years_to_work(df, *args):
    years_dict = {}
    for year in args:
        years_dict[year] = df[df['year'] == year]
    return pd.concat(years_dict).droplevel(0)

df_data_by_year = choose_years_to_work(df_normalized_teams, 2021)
df_data_by_year.drop('start', axis=1, inplace=True)

# [SKIP] Retrieve the champions stats given a champion id and merge with the BLUE/RED team

From the page https://ddragon.leagueoflegends.com/cdn/12.3.1/data/en_US/champion.json, get the stats of a champion given a key or id value.

In [140]:
'''champion_description = '../raw_data/lol_12_3_1.json'

with open(champion_description) as data_file:    
    data_champions = json.load(data_file)'''

"champion_description = '../raw_data/lol_12_3_1.json'\n\nwith open(champion_description) as data_file:    \n    data_champions = json.load(data_file)"

In [141]:
# get the key and the stats for every champion in this version
# create a new dictionary with only the values of interest

'''champions_dict = {}
champions_names = list(data_champions['data'].keys())
for champion in champions_names:
    champions_dict[int(data_champions['data'][champion]['key'])] = data_champions['data'][champion]['stats']
champions_df = pd.DataFrame(champions_dict)
champions_df = champions_df.T
champions_df.reset_index(level=0, inplace=True)
champions_df['champion'] = champions_df['index']
champions_df.drop('index', axis=1, inplace=True)
champions_df'''

"champions_dict = {}\nchampions_names = list(data_champions['data'].keys())\nfor champion in champions_names:\n    champions_dict[int(data_champions['data'][champion]['key'])] = data_champions['data'][champion]['stats']\nchampions_df = pd.DataFrame(champions_dict)\nchampions_df = champions_df.T\nchampions_df.reset_index(level=0, inplace=True)\nchampions_df['champion'] = champions_df['index']\nchampions_df.drop('index', axis=1, inplace=True)\nchampions_df"

In [142]:
#Merge with BLUE team and drop the id of the champion, no needed anymore
'''df_BLUE = pd.merge(df_normalized_BLUE, champions_df, left_on='champion_id', right_on='champion', how='left')
df_BLUE.drop(['champion', 'champion_id'], axis=1, inplace=True)'''

"df_BLUE = pd.merge(df_normalized_BLUE, champions_df, left_on='champion_id', right_on='champion', how='left')\ndf_BLUE.drop(['champion', 'champion_id'], axis=1, inplace=True)"

In [143]:
#Merge with RED team and drop the id of the champion, no needed anymore
'''df_RED = pd.merge(df_normalized_RED, champions_df, left_on='champion_id', right_on='champion', how='left')
df_RED.drop(['champion', 'champion_id'], axis=1, inplace=True)'''

"df_RED = pd.merge(df_normalized_RED, champions_df, left_on='champion_id', right_on='champion', how='left')\ndf_RED.drop(['champion', 'champion_id'], axis=1, inplace=True)"

In [144]:
# do some feature engineering

# [Skip] Merged Blue and Red data

df_BLUE_RED = pd.merge(df_BLUE, df_RED, left_on='game_id', right_on='game_id', how='left')
df_BLUE_RED.sample(5)

df_BLUE_RED.keys()

df_BLUE_RED.info()

from sklearn.preprocessing import LabelEncoder

y = LabelEncoder().fit(df_BLUE_RED.win_x).transform(df_BLUE_RED.win_x)
y.sum(), y.sum()/len(y)

df_BLUE_RED.drop(['name_x', 'id_x', 'champion_id_x', 'win_x', 'side_x', 'game_id', 'name_y', 'id_y', 'champion_id_y', 'win_y', 'side_y'], axis=1, inplace=True)

df_BLUE_RED.keys()

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')
ohe.fit(df_BLUE_RED[['role_x']])
blue_role_encoded = ohe.transform(df_BLUE_RED[['role_x']])
red_role_encoded = ohe.transform(df_BLUE_RED[['role_y']])
ohe.categories_

df_BLUE_RED['Blue_BOT'], df_BLUE_RED['Blue_JGL'], df_BLUE_RED['Blue_MID'], df_BLUE_RED['Blue_SUP'], df_BLUE_RED['Blue_TOP'] = blue_role_encoded.T
df_BLUE_RED.drop('role_x', axis=1, inplace=True)
df_BLUE_RED.sample(5)

df_BLUE_RED['Red_BOT'], df_BLUE_RED['Red_JGL'], df_BLUE_RED['Red_MID'], df_BLUE_RED['Red_SUP'], df_BLUE_RED['Red_TOP'] = red_role_encoded.T
df_BLUE_RED.drop('role_y', axis=1, inplace=True)
df_BLUE_RED.sample(5)

ohe_champ = OneHotEncoder(sparse = False, handle_unknown='ignore')
ohe_champ.fit(df_BLUE_RED[['champion_name_x']])
blue_champ_role_encoded = ohe_champ.transform(df_BLUE_RED[['champion_name_x']])
red_champ_role_encoded = ohe_champ.transform(df_BLUE_RED[['champion_name_y']])
ohe_champ.categories_

df_BLUE_RED['Blue_BOT'], df_BLUE_RED['Blue_JGL'], df_BLUE_RED['Blue_MID'], df_BLUE_RED['Blue_SUP'], df_BLUE_RED['Blue_TOP'] = blue_role_encoded.T
df_BLUE_RED.drop('role_x', axis=1, inplace=True)
df_BLUE_RED.sample(5)

# ML model

In [165]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

# Impute then Scale for numerical variables: 
num_transformer = make_pipeline(
                    SimpleImputer(strategy = 'mean'),
                    MinMaxScaler())

# Encode categorical variables
cat_transformer = OneHotEncoder(sparse = False, handle_unknown='ignore')

# Paralellize "num_transformer" and "One hot encoder"
preproc = make_column_transformer(
    (num_transformer, make_column_selector(dtype_include=['float64'])),
    (cat_transformer, make_column_selector(dtype_include=['object','bool'])),
    remainder='passthrough')

#add model
pipe = make_pipeline(preproc, LogisticRegression(solver='liblinear'))
pipe

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('pipeline',
                                                  Pipeline(memory=None,
                                                           steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                        

In [161]:
from sklearn.preprocessing import LabelEncoder

y = LabelEncoder().fit(df_BLUE.win).transform(df_BLUE.win)
X = df_BLUE.drop(['name', 'id', 'win', 'champion_id', 'side', 'game_id'], axis=1)

In [162]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [163]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118322 entries, 172499 to 121958
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   role                 118322 non-null  object 
 1   champion_name        118322 non-null  object 
 2   gold_15              118271 non-null  float64
 3   kills_assists_15     118306 non-null  float64
 4   deaths_15            118306 non-null  float64
 5   total_gold           118322 non-null  int64  
 6   total_cs             118322 non-null  int64  
 7   total_kills          118322 non-null  int64  
 8   total_monster_kills  118322 non-null  int64  
 9   total_assists        118322 non-null  int64  
 10  total_deaths         118322 non-null  int64  
 11  total_damage_taken   118322 non-null  int64  
 12  total_damage_dealt   118322 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 12.6+ MB


In [167]:
# Train pipeline
pipe.fit(X_train,y_train)

# Make predictions
pipe.predict(X_test.iloc[0:2])

# Score model
pipe.score(X_test,y_test)

0.8634132948968736

In [169]:
from sklearn.model_selection import cross_val_score

# Cross validate pipeline
cross_val_score(pipe, X_train, y_train, cv=10, scoring='r2').mean()

0.44794369360295294